#We go through below:

1. Data Cleaning and Deduplication
2. why do we convert text to vector
3. Bag of words
4. Text processing stemming
5. uni-gram, bi-gram and n-grams.
6. tf-idf
7. why tf-idf
8. word2Vec etc.


###Load Dataset

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import string
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
co = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/Amazon-fine-food-reviews/database.sqlite')

In [7]:
filtered_data = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3""",co)

In [8]:
#hrer categorizing the data into positives and negatives
def partition(x):
  if x<3:
    return 'negative'
  return 'positive'

#below I am changing the reviews of scores less than 3 to negative and greater to positive

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative


In [9]:
filtered_data.shape

(525814, 10)

In [10]:
filtered_data.head()

#from the below output we can see that the score column is changed to positives and negatives

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


### 1. Basic Data Cleaning and Deduplication

In [11]:
filtered_data.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Time
count,525814.000000,525814.000000,525814.000000,5.258140e+05
mean,284599.060038,1.747293,2.209544,1.295943e+09
std,163984.038077,7.575819,8.195329,4.828129e+07
min,1.000000,0.000000,0.000000,9.393408e+08
25%,142730.250000,0.000000,0.000000,1.270598e+09
50%,284989.500000,0.000000,1.000000,1.310861e+09
75%,426446.750000,2.000000,2.000000,1.332634e+09
max,568454.000000,866.000000,878.000000,1.351210e+09


In [12]:
filtered_data.nunique()

# we can see we have 5lakhs data but there are only 2lakhs unique User id's which
# means we have duplicate data.

Id                        525814
ProductId                  72005
UserId                    243414
ProfileName               208273
HelpfulnessNumerator         222
HelpfulnessDenominator       227
Score                          2
Time                        3157
Summary                   271924
Text                      363836
dtype: int64

In [13]:
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [14]:
display = pd.read_sql_query("""SELECT * FROM  Reviews WHERE Score !=3 AND UserId = "AR5J8UI46CURR" ORDER BY ProductID""",co)

In [15]:
display

#from the below outut we can see everything is similar except productID which means amazon is reusing the same
# reviews to the similar products of it like weights of it might be different, flavour might be different but the same reviews
# is being populated to all of them.

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [16]:
display1 = pd.read_sql_query("""SELECT * FROM  Reviews WHERE Score !=3 AND UserId = "ABXLMWJIXXAIN" ORDER BY ProductID""",co)

In [17]:
display1

#for the below output we can see the user is having different reviews for each product.


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,320691,B000CQ26E0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",0,0,4,1187740800,"Fast, Easy and organic","For speed and wholesome goodness, Annie's can ..."
1,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
2,468954,B004DMGQKE,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",0,0,5,1351209600,Awesome service and great products,We sent this product as a gift to my husband's...


#### removing the duplicates

In [18]:
# first I will sort the data
sort_data = filtered_data.sort_values('ProductId',axis = 0, ascending= True )

In [19]:
# dropping the duplicates
final_data = sort_data.drop_duplicates(subset= {'UserId','ProfileName','Time','Text'}, keep = 'first', inplace = False)
final_data.shape

(364173, 10)

In [20]:
# Checking how much percentage of data we have
(final_data['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [21]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score !=3 AND Id = 44737 OR Id = 64422 ORDER BY ProductID""",co)
display

## from the below output we can see helpfulness numerator is greater than demominator
# but it shouldnt be because helpfulness numerator is always lessthan = denominator.
# so we remove this kind of data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [22]:
final = final_data[final_data.HelpfulnessNumerator <= final_data.HelpfulnessDenominator]


In [23]:
print(final.shape)

(364171, 10)


In [24]:
#find howmany positive reviews and negative reviews are present in our dataset?
final['Score'].value_counts()

##from the below ouptut we have positives more than negatives.

Score
positive    307061
negative     57110
Name: count, dtype: int64

##### 2. Why convert text to vector
##### 3. Bag of words
##### 4. Text processing stemming
##### 5. uni-gram, bi-gram and n-grams.
##### 6. tf-idf
##### 7. why tf-idf
##### 8. word2Vec etc.


Refer to Lecture written notes

In [25]:
#refer to lecture notes.

##2.Bag of words code implementation

In [26]:
#BoW

count_vect = CountVectorizer()
final_count = count_vect.fit_transform(final['Text'].values)

In [27]:
type(final_count)

''' from the below output -
what is sparse matrix : if i have n X m rows and columns  in BoW most of the
of any row ri is having value zero.

for each row ri: we have time complexity high if we store the values zero..
for each row ri if we store particular column which has values and their
addresses in a dictionary then our time complexity will be lil bit lower.'''

' from the below output - \nwhat is sparse matrix : if i have n X m rows and columns  in BoW most of the \nof any row ri is having value zero.\n\nfor each row ri: we have time complexity high if we store the values zero..\nfor each row ri if we store particular column which has values and their\naddresses in a dictionary then our time complexity will be lil bit lower.'

In [28]:

final_count.get_shape()

(364171, 115281)

##Text preprocessing - stemming, stopword removal and lemmatization.



Preprocessing phases:
1. Begin by removing html tags.
2. Remove any punctuations or limited set of special characters like, or . or # etc.
3. check if word is made up of english letters and not alpha numeric.
4. check to see if length of word is greater than 2.
5. convert to lowercase
6. remove stop words
7. finally snowball stemming the word

In [29]:
# find sentences containing html tags
import re
i=0
for sent in final['Text'].values:
  if(len(re.findall('<.*?',sent))):
    print(i)
    print(sent)
    break
  i+=1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [31]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('stopwords')

stop = set(stopwords.words('english')) #setting stop words
sno = nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
  cleanr = re.compile('<.*?') # anything like <.*?etc compile and substitute sentence.
  cleantext = re.sub(cleanr,' ',sentence)
  return cleantext
def cleanpunc(sentence):
  cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
  cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
  return cleaned
print(stop)
print("*****************************")
print(sno.stem('tasty'))

{'on', "doesn't", 'your', 'our', 'yourselves', "couldn't", 'under', 'shan', "haven't", 'out', 'll', "wouldn't", 'ma', 'only', 'there', 'too', 'than', 'be', 'hers', 'don', 'you', 'as', 'they', 'these', 'does', 'from', 'have', 'ourselves', 'hasn', 'each', "weren't", 'if', 'at', 's', 'weren', 'so', 'once', "mustn't", 'having', "shouldn't", 'further', 'same', 'do', 'her', 'theirs', 'itself', 'me', 'is', "you'll", 'but', 'very', 'been', "hadn't", 'which', "needn't", 'because', 'below', 'mightn', 'himself', 'after', 'we', 'themselves', "mightn't", 'few', 'up', 'above', 'of', 'until', 'again', 'are', 'wasn', 'couldn', 'his', "aren't", 'or', "wasn't", 'through', 're', 'the', 'other', 'most', 'doesn', 'shouldn', 'yours', 'to', 'who', 'against', 'herself', 'he', 'him', 'all', 'for', 'and', 'some', 'where', "you've", 'now', 'during', 'yourself', 'between', 'am', 'o', 'she', "won't", 'm', 'both', 'was', 'will', 'myself', 'my', 'while', 'how', 'them', "should've", 'haven', 'no', 'before', 'by', 'sh

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [32]:
'''code to implement step by step that checks mentioned in preprocessing
this code takes a while to run. checkout lecture written notes'''

i = 0
str1 = ' '
final_string = []
all_positive_words = []
all_negative_words = []

s = ''
for sent in final['Text'].values:
  filtered_sentence = []

  sent = cleanhtml(sent)
  for w in sent.split():
    for cleaned_words in cleanpunc(w).split():
      if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):
        if(cleaned_words.lower() not in stop):
          s = (sno.stem(cleaned_words.lower())).encode('utf8')
          filtered_sentence.append(s)
          if(final['Score'].values)[i] == 'positive':
            all_positive_words.append(s)
          if(final['Score'].values)[i] == 'negative':
            all_negative_words.append(s)
        else:
          continue
      else:
        continue
  str1 = b" ".join(filtered_sentence)
  final_string.append(str1)
  i+=1


In [33]:
final['CleanedText'] = final_string

<ipython-input-33-3f42d3fec722>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['CleanedText'] = final_string


In [34]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month year goe month cute littl ...


In [35]:
#store final table into sqllite table for future

conn = sqlite3.connect('final.sqlite')
c = conn.cursor()
conn.text_factory = str
final.to_sql('Reviews',conn, schema = None, if_exists = 'replace')

364171

## Bi-Grams and N-Grams

In [36]:
freq_dist_positive = nltk.FreqDist(all_positive_words)
freq_dist_negative = nltk.FreqDist(all_negative_words)
print('Most common Positive words: ',freq_dist_positive.most_common(20))
print('Most common Negative words: ',freq_dist_negative.most_common(20))


Most common Positive words:  [(b'like', 139150), (b'tast', 128631), (b'good', 112216), (b'flavor', 109473), (b'love', 107034), (b'use', 103627), (b'great', 102818), (b'product', 99504), (b'one', 95360), (b'tri', 86237), (b'tea', 83824), (b'coffe', 78610), (b'make', 74835), (b'get', 71962), (b'food', 64752), (b'amazon', 57832), (b'would', 55297), (b'time', 55225), (b'buy', 53903), (b'realli', 52569)]
Most common Negative words:  [(b'tast', 34489), (b'like', 32284), (b'product', 29504), (b'one', 20420), (b'flavor', 19561), (b'would', 17901), (b'tri', 17676), (b'use', 15275), (b'good', 14977), (b'coffe', 14677), (b'get', 13758), (b'buy', 13690), (b'order', 12846), (b'food', 12742), (b'dont', 11683), (b'tea', 11657), (b'amazon', 11258), (b'even', 10983), (b'box', 10841), (b'make', 9816)]


In [37]:
#bi-gram, tri-gram and n-gram


# we need to remove stop words like "not" should be avoided before n-grams applyng

count_vect = CountVectorizer(ngram_range=(1,2)) #check documentation in skitlearn
final_bigram_count = count_vect.fit_transform(final['Text'].values)

In [38]:
final_bigram_count.get_shape()
'''from below output we have 2.9 lakh dimensions when i used 2 grams in
ngram_range = 1,2) if we increase 2 to 3 or 4 it will give more dimensions
which will take lot of time to run the code'''

(364171, 2910192)

## TF-IDF

In [39]:

tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(final['Text'].values)

In [40]:
final_tf_idf.get_shape()


(364171, 2910192)

In [47]:
features = tf_idf_vect.get_feature_names_out()
len(features) # it is nothing but columns

2910192

In [49]:
features[100000:100010]

array(['ales until', 'ales ve', 'ales would', 'ales you', 'alessandra',
       'alessandra ambrosia', 'alessi', 'alessi added', 'alessi also',
       'alessi and'], dtype=object)

In [51]:
#we need to convert a row in sparse matrix to a numpy array

print(final_tf_idf[3,:].toarray()[0])


[0. 0. 0. ... 0. 0. 0.]


In [52]:
#search codes that will help you like below.
# search for codes like, I need top features

def top_tfidf_feats(row,features, top_n=25):
  '''it get top n tfidf values in row and return them with correspoinding
  ranks'''
  topn_ids = np.argsort(row)[::-1][:top_n]
  top_feats =[(features[i],row[i]) for i in topn_ids]
  df = pd.DataFrame(top_feats)
  df.columns = ['feature','tfidf']
  return df

top_tfidf = top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

''' in the above top_tfidf variable we are taking review one and converting to
an array and taking top 25 features'''

In [54]:
top_tfidf

,feature,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


## Word2Vec

In [58]:
from gensim.models import Word2Vec
from gensim.models import keyedvectors
import pickle


'''in this we use pretrained model by google.
its 3gb file and occupies lot of space.
we will provide a pickle file which contains a dictionary.
it contains all our courpus words as keys and model as values
to use code snippet download googlenews-vectors-negative300.bin'''

model = keyedvectors.load_word2vec_format('GoogleNews-vectors-negative300.bin')

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 46: character maps to <undefined>

In [59]:
# train our own word2Vec model using our own text corpus

import gensim
i =0
list_of_sent = []
for sent in final['Text'].values:
  filtered_sentence = []
  sent = cleanhtml(sent)
  for w in sent.split():
    for cleaned_words in cleanpunc(w).split(): #removing punctuations
      if(cleaned_words.isalpha()): #ensuring everything is alphanumeric
        filtered_sentence.append(cleaned_words.lower()) #convertng to lowercase
      else:
        continue
  list_of_sent.append(filtered_sentence)

In [60]:
print(final['Text'].values[0])
print("*****************************************")
print(list_of_sent[0])


this witty little book makes my son laugh at loud. i recite it in the car as we're driving along and he always can sing the refrain. he's learned about whales, India, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will STILL be able to recite from memory when he is  in college
*****************************************
['this', 'witty', 'little', 'book', 'makes', 'my', 'son', 'laugh', 'at', 'loud', 'i', 'recite', 'it', 'in', 'the', 'car', 'as', 'were', 'driving', 'along', 'and', 'he', 'always', 'can', 'sing', 'the', 'refrain', 'hes', 'learned', 'about', 'whales', 'india', 'drooping', 'i', 'love', 'all', 'the', 'new', 'words', 'this', 'book', 'introduces', 'and', 'the', 'silliness', 'of', 'it', 'all', 'this', 'is', 'a', 'classic', 'book', 'i', 'am', 'willing', 'to', 'bet', 'my', 'son', 'will', 'still', 'be', 'able', 'to', 'recite', 'from', 'memory', 'when', 'he', 'is', 'in', 'college']


In [65]:
#trainign w2v model

w2v_model = gensim.models.Word2Vec(list_of_sent,min_count=5,vector_size=50,workers = 4)

''' min_count says - if a word doesnt occur atleast 5 times dont construct word2vec for it
 vector size = dimensionality of vi i.e., we are giving 50 dimensions'''


' min_count says - if a word doesnt occur atleast 5 times dont construct word2vec for it\n vector size = dimensionality of vi i.e., we are giving 50 dimensions'

In [67]:
# Access the vocabulary using the key_to_index dictionary
words = list(w2v_model.wv.key_to_index.keys())

# or access the vocabulary using the index_to_key list
words = [w2v_model.wv.index_to_key[i] for i in range(w2v_model.wv.vectors.shape[0])]

In [68]:
w2v_model.wv.most_similar('tasty')


[('tastey', 0.914087176322937),
 ('yummy', 0.8597651124000549),
 ('satisfying', 0.8454764485359192),
 ('filling', 0.8240659236907959),
 ('delicious', 0.816752552986145),
 ('flavorful', 0.7981603741645813),
 ('addicting', 0.771507978439331),
 ('nutritious', 0.7629421949386597),
 ('delish', 0.7538960576057434),
 ('versatile', 0.7466045618057251)]

In [69]:
w2v_model.wv.most_similar('like')


[('resemble', 0.7077288627624512),
 ('dislike', 0.6601505875587463),
 ('prefer', 0.6547054052352905),
 ('mean', 0.6442059278488159),
 ('think', 0.6099909543991089),
 ('overpower', 0.6089782118797302),
 ('enjoy', 0.60748690366745),
 ('love', 0.5743322968482971),
 ('gross', 0.573230504989624),
 ('overwhelm', 0.5712978839874268)]

In [73]:
count_vect_feat = count_vect.get_feature_names_out().tolist()  # Convert the NumPy array to a list
index = count_vect_feat.index('like')  # Find the index of the word 'like'
print(count_vect_feat[index])

like


## Avg w2V, TFIDF-w2v

In [76]:
#compute average w2v for each review
sent_vectors = []
for sent in list_of_sent:
    sent_vec = np.zeros(50)  # as word vec are of zero length
    cnt_words = 0
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec/=cnt_words
    sent_vectors.append(sent_vec)  # Add the sentence vector to the list
print(len(sent_vectors))
print(len(sent_vectors[0]))

<ipython-input-76-e5b9374c1b0c>:13: RuntimeWarning: divide by zero encountered in divide
  sent_vec/=cnt_words
<ipython-input-76-e5b9374c1b0c>:13: RuntimeWarning: invalid value encountered in divide
  sent_vec/=cnt_words


364171
50


In [77]:
# TF-IDF weighted word2vec

tfidf_feat = tf_idf_vect.get_feature_names_out()

tfidf_sent_vectors = []
row = 0

for sent in list_of_sent:
  sent_vec = np.zeros(50)
  weight_sum = 0
  for word in sent:
    try:
      vec = w2v_model.wv[word]
      tfidf = final_tf_idf[row,tfidf_feat.index(word)]
      sent_vec += (vec*tf_idf)
      weight_sum += tf_idf
    except:
      pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1


<ipython-input-77-46aee1e5ba8b>:19: RuntimeWarning: divide by zero encountered in divide
  sent_vec /= weight_sum
<ipython-input-77-46aee1e5ba8b>:19: RuntimeWarning: invalid value encountered in divide
  sent_vec /= weight_sum


In [78]:
print(len(tfidf_sent_vectors))
print(len(tfidf_sent_vectors[0]))

28128140
50
